In [1]:
# CTA_STEPS 0, 1, and 2
# Step 0: Data Cleaning
# Step 1: Predict outcome
# Step 2: Compute SHAP values

In [2]:
#import packages
import re
import numpy as np
import pandas as pd
import random

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess

# spacy for lemmatization
import spacy

# Plotting tools

import matplotlib.pyplot as plt


# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)



from gensim.models import Word2Vec


import random

import statsmodels.api as sm
import shap

from nltk.tokenize import RegexpTokenizer

from nltk.tokenize.treebank import TreebankWordDetokenizer


import copy
from gensim.models import KeyedVectors

from itertools import combinations

import nltk

from itertools import combinations
from itertools import product

from sklearn.feature_extraction.text import CountVectorizer

from gensim.models import FastText

from scipy.stats import sem
import scipy as sp

from scipy import stats

In [3]:
# Step 0: data cleaning

In [4]:
np.random.seed(42)
random.seed(42)

The dataset for the following example was also used by Roberts et al. (2014) to validate the Structural Topic Model (STM), and it is available on Harvard Dataverse. It is called "gadarian_metadata.csv" and is available under https://doi.org/10.7910/DVN/29405.

In [5]:
# load data
path_data=""
df = pd.read_csv(path_data+"final_anes_metadata.csv",sep=",",dtype=str)

df.head(5)

,name,path,age,caseid,female,highest grade completed,ideology,independents,mii_1,mii_2,...,mippol2_substantive3,mippol2_substantive4,mippol2_substantive5,mippol2_substantive6,mippol2_substantive7,mippol2_substantive8,pid,pid_strong,pid_summary,txtorg_id
0,NaN,NaN,35,1,1,13,1,1,the economy is first,the 2 wars,...,NaN,NaN,NaN,NaN,NaN,NaN,3,-1,4,3de40634-3386-11e3-a3e4-88532e617cea
1,NaN,NaN,39,3,2,16,2,-1,abortion,gay marriage,...,NaN,NaN,NaN,NaN,NaN,NaN,2,5,5,3deba592-3386-11e3-a3e4-88532e617cea
2,NaN,NaN,50,4,1,12,1,3,economy,more jobs,...,NaN,NaN,NaN,NaN,NaN,NaN,5,-1,3,3dede46a-3386-11e3-a3e4-88532e617cea
3,NaN,NaN,72,5,1,16,1,-1,character//no,//experience//no,...,NaN,NaN,NaN,NaN,NaN,NaN,2,1,6,3df03382-3386-11e3-a3e4-88532e617cea
4,NaN,NaN,66,7,2,16,2,1,experience,"a record of what the person has done, or actua...",...,NaN,NaN,NaN,NaN,NaN,NaN,3,-1,4,3df1d52a-3386-11e3-a3e4-88532e617cea


In [9]:
# remove rows with ideology (-4)
# identify text and ourcome
df['ideology']=pd.to_numeric(df['ideology'])
df['pid_summary']=pd.to_numeric(df['pid_summary'])
df=df[df['pid_summary']>0]
df=df[df['pid_summary']!=3].reset_index(drop=True)
df['republican']=[1 if x>3 else 0 for x in df['pid_summary']]
#text=df['mii_1']
text=df['mip_1']
#outcome=df['ideology']-1
outcome=df['republican']
text_original=df['mip_1']
metadata=df[['miiele1_code1','mippol1_code1','age','female']]

In [14]:
def sent_to_words_orig(sentences):
    for sentence in sentences:
        yield(str(sentence).split())

# remove stopwords

# Load the spacy model (for other languages, use another spacy model)
nlp = spacy.load('en_core_web_sm')

text = text.apply(lambda x: ' '.join([token.text for token in nlp(x) if not token.is_stop]))

# lemmatize text (optional)

text_lemma = text.apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))

df_tosave=pd.DataFrame({'text':text_lemma,'republican':outcome, 'text_raw':text, 'text_original':text_original})
df_tosave = pd.concat([df_tosave, metadata], axis=1)
df_tosave = df_tosave[df_tosave['text'].str.len() > 0].reset_index(drop=True)

text_lemma=df_tosave['text']
outcome=df_tosave['republican']
text_orig=df_tosave['text_raw']


words = text_lemma.apply(nltk.word_tokenize)
words_orig = text_orig.apply(nltk.word_tokenize)

# Flatten the list of words
words = [word for sublist in words for word in sublist]
words_orig = [word for sublist in words_orig for word in sublist]

# Create a frequency distribution
freqdist = nltk.FreqDist(words)

sorted_freqdist = sorted(freqdist.items(), key=lambda x: x[1],reverse=True)

# Create a frequency distribution
freqdist_orig = nltk.FreqDist(words_orig)

sorted_freqdist_orig = sorted(freqdist_orig.items(), key=lambda x: x[1],reverse=True)

# Define minimum number of characters
min_characters = 3

# Assuming text_lemma is your lemmatized text
vectorizer = CountVectorizer(min_df=1, token_pattern=r'\b\w{%d,}\b' % min_characters)

# fit the vectorizer on the lemmatized text
X = vectorizer.fit_transform(text_lemma)

# X is a sparse matrix representing the bag-of-words model
# To get the feature names (words), you can use
feature_names = vectorizer.get_feature_names_out()

# To convert the matrix into a DataFrame:
# Here, feature_names (which are your words) are used as column names in the DataFrame
bow_df = pd.DataFrame(X.toarray(), columns=feature_names)

data_words = list(sent_to_words_orig(text_lemma))

data_words_orig = list(sent_to_words_orig(text_orig))

In [17]:
len(feature_names)

638

In [8]:
# Step 1: predict outcome

In [9]:
# define text and outcome for BERT

text_for_bert=df_tosave['text']
labels_for_bert=df_tosave['treatment']

In [10]:
# Predict outcome with BERT

from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
import numpy as np
import random

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

set_seed(42)  # You can use any seed you want

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Assume you have a DataFrame 'df' with 'text_lemma' as text column and 'outcome' as label
texts = text_for_bert.tolist()
labels = pd.to_numeric(labels_for_bert).tolist()

inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
labels = torch.tensor(labels)

# Create a DataLoader
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
dataloader = DataLoader(dataset, batch_size=16)

# Choose an optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

all_labels = []
all_predictions = []

# Fine-tuning loop
for epoch in range(20):
    for batch in dataloader:
        optimizer.zero_grad()
        
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()

        # Get predictions for each batch
        preds = torch.argmax(outputs.logits, dim=-1)
        all_predictions.extend(preds.tolist())
        all_labels.extend(labels.tolist())





Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning


In [11]:
# Step 2: obtain feature importance

In [ ]:
# Use SHAP to obtain feature importance

# Define a prediction function, use CPU
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding='max_length', max_length=512, truncation=True) for v in x])
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:,1]) # use one vs rest logit units
    return val

# Create an explainer object
explainer = shap.Explainer(f, tokenizer)

# Explain the model's predictions on text
shap_values = explainer(text_for_bert, fixed_context=1)


In [12]:
# identify words that could be allocated to an embedding (important for Step 3)
unique_words = list(set(words))
unique_words_filtered = [word for word in unique_words if len(word) >= 3]

In [13]:
# make a dataframe out of the SHAP values lists

shap_allwords=pd.DataFrame(np.zeros((df_tosave.shape[0], len(unique_words_filtered))))
shap_allwords.columns=unique_words_filtered

index=0
for i in range(text_lemma.shape[0]):
    words_tmp=shap_values[i].data
    shap_tmp=shap_values[i].values
    for j in range(len(words_tmp)):
        word_tmp=words_tmp[j].lstrip()
        if word_tmp in unique_words_filtered:
            shap_allwords[word_tmp][i]=shap_tmp[j]
            index=index+1
#             print(index)
shap_allwords.shape      

(1297, 643)

In [16]:
#save SHAP values

shap_allwords.to_csv("anes_shap_bert_allwords_110324_train_lemma_shap_lemma.csv", index=False)